In [1]:
# TODO: implement 2 methods of extracting key phrase in a query: tf-idf based and prompt based.

In [2]:
import jieba.analyse
import json

In [3]:
questions = json.load(open('./questions.json'))

In [26]:
# tfidf based keyword extraction

tfidf_keywords_list = []

for i in range(len(questions)):
    query = questions[i]['question']
    keywords = jieba.analyse.extract_tags(query, topK=3)
    tfidf_keywords_list.append(keywords)

In [27]:
tfidf_keywords_list

[['第几页', '前排', '座椅'],
 ['固定装置', '座椅', '一页'],
 ['前机', '第几页', '舱盖'],
 ['前机', '舱盖', '一页'],
 ['第几页', '记录仪', '行车'],
 ['记录', 'Lynk', 'Co'],
 ['EDR', '黑客', '数据'],
 ['环保法', '报废', '哪些'],
 ['车辆', '互联', '报废'],
 ['万无一失', '用车', '确保'],
 ['储物', '装载', '车内'],
 ['手套', '请问', '容量'],
 ['储物', '合理安排', '实用性'],
 ['手套', '请问', '容量'],
 ['储物袋', '靠背', '前排'],
 ['扶手', '后排', '占用'],
 ['解锁', '验证码', '手套'],
 ['锁止', '手套', '密码'],
 ['尾门', '关闭', '确保'],
 ['尾门', '障碍物', '感应'],
 ['尾门', '安全可靠', '关闭'],
 ['尾门', '障碍物', '关闭'],
 ['尾门', '伤及', '障碍物'],
 ['关闭', '尾门', '中触'],
 ['装载', '安全性', '货物'],
 ['装载', '货物', '这辆'],
 ['解锁', '上锁', '遥控'],
 ['遥控', '钥匙', '干扰'],
 ['闭锁', '解锁', '触摸'],
 ['钥匙', '解锁', '蓝牙'],
 ['预警系统', '低速', '静止'],
 ['主驾', '下会', '迎宾'],
 ['防盗', '确保', '车辆'],
 ['确保安全', '座椅', '驾驶员'],
 ['踏板', '制动', '座椅'],
 ['座椅', '记忆', '正确'],
 ['座椅', '移动', '手动'],
 ['显示屏', '界面', '方向盘'],
 ['方向盘', '请问', '倾斜'],
 ['按键', '方向盘', '请问'],
 ['方向盘', '请问', '倾斜'],
 ['哪项', '方向盘', '警告'],
 ['柱未', '锁止', '驾车'],
 ['胎压', '报警', '闪烁'],
 ['漏气', '紧急措施', '确保安全'],
 ['雨刷', '正确', '自动

In [28]:
from collections import Counter

keywords_cnt = Counter()
for x in tfidf_keywords_list:
    keywords_cnt.update(x)

In [29]:
keywords_cnt.most_common(n=5)

[('充电', 21), ('请问', 20), ('下会', 17), ('辅助', 15), ('Lynk', 14)]

In [10]:
import time
import jwt
import requests

In [11]:
KEY = '83e5bc58555d8bac289e27bac50f8afc.Khk1JjCxb8MJN8Mi'

# 实际KEY，过期时间



def generate_token(apikey: str, exp_seconds: int):
    try:
        id, secret = apikey.split('.')
    except Exception as e:
        raise Exception('invalid apikey', e)

    payload = {
        'api_key': id,
        'exp': int(round(time.time() * 1000)) + exp_seconds * 1000,
        'timestamp': int(round(time.time() * 1000)),
    }
    return jwt.encode(
        payload,
        secret,
        algorithm='HS256',
        headers={'alg': 'HS256', 'sign_type': 'SIGN'},
    )

In [12]:
def ask_glm(content):
    url = "https://open.bigmodel.cn/api/paas/v4/chat/completions"
    headers = {
      'Content-Type': 'application/json',
      'Authorization': generate_token(KEY, 1000)
    }

    data = {
        "model": "glm-3-turbo",
        "messages": [{"role": "user", "content": content}]
    }

    response = requests.post(url, headers=headers, json=data)
    return response.json()

In [13]:
ask_glm('你好，请讲一个苏联笑话。')

{'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': '好的，这是一个苏联时期的笑话：\n\n有一天，一个苏联公民走进了一个商店。他看到货架上有一个标着“应有尽有”的柜子。于是，他好奇地问店员：“这个柜子里真的什么都有吗？”\n\n店员回答说：“是的，里面应有尽有。”\n\n公民想了一下，然后问：“那好，请问我能从这里买一公斤面包吗？”\n\n店员回答：“对不起，这里没有面包。”\n\n这个笑话反映了苏联时期物资短缺的现实，同时也展示了人们对于“应有尽有”这个概念的幽默解读。',
    'role': 'assistant'}}],
 'created': 1708274757,
 'id': '8367747687520561003',
 'model': 'glm-3-turbo',
 'request_id': '8367747687520561003',
 'usage': {'completion_tokens': 128, 'prompt_tokens': 13, 'total_tokens': 141}}

In [23]:
prompt = '你是一个专业客服，需要准确识别用户提问的关键词方便检索。请提取以下提问中的关键词: {}'

In [24]:
prompt.format('1+1等于几')

'你是一个专业客服，需要准确识别用户提问的关键词方便检索。请提取以下提问中的关键词: 1+1等于几'

In [25]:
ask_glm(prompt.format('Zara、H&M、GAP 和优衣库各自的竞争优势是什么？ - 曹暘暘的回答'))

{'choices': [{'finish_reason': 'stop',
   'index': 0,
   'message': {'content': '用户提问中的关键词包括：Zara、H&M、GAP、优衣库、竞争优势。这些关键词将帮助您定位到相应的零售品牌，并了解它们在市场中的比较优势。',
    'role': 'assistant'}}],
 'created': 1708275145,
 'id': '8367746175691918512',
 'model': 'glm-3-turbo',
 'request_id': '8367746175691918512',
 'usage': {'completion_tokens': 45, 'prompt_tokens': 52, 'total_tokens': 97}}

In [31]:
prompt_keywords_list = []

for i in range(len(questions)):
    query = questions[i]['question']
    prompt_keywords_list.append(ask_glm(prompt.format(query)))

In [32]:
prompt_keywords_list

[{'choices': [{'finish_reason': 'stop',
    'index': 0,
    'message': {'content': '关键词: “前排座椅通风”', 'role': 'assistant'}}],
  'created': 1708275281,
  'id': '8370007802391121168',
  'model': 'glm-3-turbo',
  'request_id': '8370007802391121168',
  'usage': {'completion_tokens': 9, 'prompt_tokens': 38, 'total_tokens': 47}},
 {'choices': [{'finish_reason': 'stop',
    'index': 0,
    'message': {'content': '关键词：儿童安全座椅固定装置、车辆、相关内容、页面。',
     'role': 'assistant'}}],
  'created': 1708275282,
  'id': '8367747447002354137',
  'model': 'glm-3-turbo',
  'request_id': '8367747447002354137',
  'usage': {'completion_tokens': 17,
   'prompt_tokens': 889,
   'total_tokens': 906}},
 {'choices': [{'finish_reason': 'stop',
    'index': 0,
    'message': {'content': '关键词: "打开前机舱盖"', 'role': 'assistant'}}],
  'created': 1708275283,
  'id': '8367747343923126265',
  'model': 'glm-3-turbo',
  'request_id': '8367747343923126265',
  'usage': {'completion_tokens': 11, 'prompt_tokens': 40, 'total_tokens': 51}},


In [34]:
prompt_keywords_list = [x['choices'][0]['message']['content'] for x in prompt_keywords_list]

In [42]:
prompt_keywords_list

['关键词: “前排座椅通风”',
 '关键词：儿童安全座椅固定装置、车辆、相关内容、页面。',
 '关键词: "打开前机舱盖"',
 '关键词: "打开前机舱盖"',
 '关键词：“行车记录仪视频”、“页数”。',
 '关键词：Lynk&Co、领克汽车、事件数据记录系统（EDR）、记录信息。',
 '关键词：事件数据记录系统（EDR）、数据、黑客、利用、恶意攻击',
 '关键词：国家环保法、车辆报废处理',
 '关键词：车辆报废、原车主、智能互联服务。',
 '关键词：确保、用车前、准备工作、万无一失。',
 '关键词：装载货物、车内、储物空间。',
 '关键词：车辆、手套箱、容量、大小',
 '关键词：合理安排、前排、储物空间、提高实用性',
 '关键词：车辆、手套箱、容量、大小',
 '关键词：如何正确使用、前排座椅靠背、储物袋',
 '关键词：后排中央扶手、杯托、占用、驾驶员、放置饮料。',
 '关键词：如何、手机验证码、解锁、手套箱。',
 '关键词：手套箱、锁止、密码、怎么办',
 '关键词: 尾门、自动打开、自动关闭、安全停止',
 '关键词: 尾门自动打开/关闭, 感应障碍物, 机制',
 '用户提问中的关键词是：“如何确保”、“尾门”、“关闭时”、“安全可靠”。这些关键词可以帮助您快速定位到用户关心的问题点，即关于尾门关闭的安全性。',
 '关键词：尾门、关闭过程、障碍物、反应',
 '关键词：电动尾门、运动、伤及、人员、障碍物。',
 '关键词：尾门、关闭过程、障碍物、自动关闭、位置',
 '关键词：装载货物、安全性',
 '关键词: 正在装载、货物、汽车、装载立方米',
 '关键词：设置、遥控钥匙、靠近解锁、远离上锁功能',
 '关键词：确保、遥控钥匙、操作、不受干扰',
 '关键词：触摸车门外把手、车内解锁、闭锁、外部感应区域、操作车辆闭锁。',
 '关键词：遥控钥匙、蓝牙钥匙、丢失、损坏、车辆、闭锁、解锁操作。',
 '关键词：车辆、低速行驶、静止、乘客、安全、开门预警系统。',
 '关键词：主驾座椅迎宾功能、自动停用。',
 '关键词：确保、车辆、防盗系统、正常工作。',
 '关键词：驾驶前、调节驾驶员座椅、确保安全、舒适。',
 '关键词：不调节驾驶员座椅、正常操作、制动踏板、加速踏板',
 '关键词：如何正确使用、座椅位置记忆功能。'

In [59]:
import re

final_prompt_keywords_list = [re.split(r':|：|，|、|。|“|”', x)[1:] for x in prompt_keywords_list]

In [60]:
final_prompt_keywords_list

[[' ', '前排座椅通风'],
 ['儿童安全座椅固定装置', '车辆', '相关内容', '页面'],
 [' "打开前机舱盖"'],
 [' "打开前机舱盖"'],
 ['行车记录仪视频', '页数'],
 ['Lynk&Co', '领克汽车', '事件数据记录系统（EDR）', '记录信息'],
 ['事件数据记录系统（EDR）', '数据', '黑客', '利用', '恶意攻击'],
 ['国家环保法', '车辆报废处理'],
 ['车辆报废', '原车主', '智能互联服务'],
 ['确保', '用车前', '准备工作', '万无一失'],
 ['装载货物', '车内', '储物空间'],
 ['车辆', '手套箱', '容量', '大小'],
 ['合理安排', '前排', '储物空间', '提高实用性'],
 ['车辆', '手套箱', '容量', '大小'],
 ['如何正确使用', '前排座椅靠背', '储物袋'],
 ['后排中央扶手', '杯托', '占用', '驾驶员', '放置饮料'],
 ['如何', '手机验证码', '解锁', '手套箱'],
 ['手套箱', '锁止', '密码', '怎么办'],
 [' 尾门', '自动打开', '自动关闭', '安全停止'],
 [' 尾门自动打开/关闭, 感应障碍物, 机制'],
 ['如何确保', '尾门', '关闭时', '安全可靠', '这些关键词可以帮助您快速定位到用户关心的问题点', '即关于尾门关闭的安全性'],
 ['尾门', '关闭过程', '障碍物', '反应'],
 ['电动尾门', '运动', '伤及', '人员', '障碍物'],
 ['尾门', '关闭过程', '障碍物', '自动关闭', '位置'],
 ['装载货物', '安全性'],
 [' 正在装载', '货物', '汽车', '装载立方米'],
 ['设置', '遥控钥匙', '靠近解锁', '远离上锁功能'],
 ['确保', '遥控钥匙', '操作', '不受干扰'],
 ['触摸车门外把手', '车内解锁', '闭锁', '外部感应区域', '操作车辆闭锁'],
 ['遥控钥匙', '蓝牙钥匙', '丢失', '损坏', '车辆', '闭锁', '解锁操作'],
 ['车辆', '低速行驶', '

In [62]:
# 比较，使用jieba的extract_tags方法可以指定关键词数目，虽然所有询问关键词的统计不好，但是每个询问的关键词还是比较合理的。使用prompt抽取关键词，需要面临几个问题：一是关键词数目难以指定，模型有时候会忽略对关键词数目的限制，二是格式难以解析，不方便后续使用。